In [4]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import sklearn.linear_model as sl
import scipy.stats as st

In [97]:
ncaa = pd.read_csv("http://www4.stat.ncsu.edu/~boos/var.select/ncaa.data2.txt", 
                   delim_whitespace = True)
x = ncaa.ix[:,:-1]
y = ncaa.ix[:,-1]
x.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19
0,13,17,9,15,28.0,0,-1.14045,3.660,4.490,3409,65.8,18,81,42.2,660000,77,100,59,1
1,28,20,32,18,18.4,18,-0.13719,2.594,3.610,7258,66.3,17,82,40.5,150555,88,94,41,25
2,32,20,20,20,34.8,18,1.55358,2.060,4.930,6405,75.0,19,71,46.5,415400,94,81,25,36
3,32,21,24,21,14.5,20,2.05712,2.887,3.876,18294,66.0,16,84,42.2,211000,93,88,26,13
4,24,20,16,20,21.8,13,-0.77082,2.565,4.960,8259,63.5,16,91,41.2,44000,90,92,32,31


In [61]:
def reg_subset(x,y):
    lm = sl.LinearRegression()
    
    in_ = []
    out_ = list(range(x.shape[1]))
    rss = []
    (n,m) = x.shape
    if(m>=n):
        ml = n-5
    else:
        ml = m   
    vm = np.array(range(ml))

    for pi in range(x.shape[1]):
        rss_find = []
        for i in out_:
            fit_X = pd.DataFrame(x.ix[:, in_ + [i]])
            lm.fit(fit_X, y)
            pred = lm.predict(fit_X)
            rss_find.append(sum((pred-y)**2))
        min_rss = np.min(rss_find)
        min_var = out_[np.argmin(rss_find)]
        rss.append(min_rss)
        in_.append(min_var)
        del out_[np.argmin(rss_find)]
        in_var = [x.columns[i] for i in in_]
    rss.insert(0,sum((y-np.mean(y))**2))
    rss = np.array(rss)
    pv_org = 1 - st.f.cdf((rss[vm] - rss[vm+1])*(n-(vm+2))/rss[vm+1],
                     1,n-(vm+2))
    return (np.array(in_),np.array(in_var),rss, pv_org)

In [62]:
reg_subset(x,y)

(array([ 1,  2,  4,  3,  6, 16, 14,  5,  8,  7, 11,  9, 12, 17, 10,  0, 13,
        18, 15]),
 array(['x2', 'x3', 'x5', 'x4', 'x7', 'x17', 'x15', 'x6', 'x9', 'x8', 'x12',
        'x10', 'x13', 'x18', 'x11', 'x1', 'x14', 'x19', 'x16'], 
       dtype='<U3'),
 array([ 24148.87276596,   7077.86138645,   5942.5387484 ,   5534.10114754,
          5068.71560461,   4565.33245127,   4354.85020305,   4167.86982143,
          4040.70513154,   3897.53072484,   3725.51704978,   3661.59293365,
          3610.27305639,   3564.96214367,   3510.07074574,   3489.07672866,
          3478.67153268,   3472.10439175,   3470.66621376,   3469.96781547]),
 array([  1.11022302e-16,   6.95109478e-05,   1.15845889e-02,
          5.29907450e-03,   2.48342482e-03,   4.33151925e-02,
          5.27341418e-02,   1.05631519e-01,   8.26270613e-02,
          5.36356789e-02,   2.34958569e-01,   2.86440303e-01,
          3.16318194e-01,   2.69726331e-01,   4.95325787e-01,
          6.32648734e-01,   7.05641795e-01,   8.605

In [129]:
def fsr_fast_pv(pv_orig, m, gam0 = 0.05, digits = 4, printout = True, plot = True):
    m1 = len(pv_orig)
    pvm = np.zeros(m1)

    # create monotone p-values
    for i in range(0, m1):
        pvm[i] = np.max(pv_orig[0:(i+1)])
    alpha = np.concatenate([np.array([0]), pvm])
    ng = len(alpha)
    
    # calculate model size
    S = np.zeros(ng)
    for j in range(1, ng):
        S[j] = sum(pvm <= alpha[j])
        
    # calculate gamma hat
    ghat = (m - S)*alpha/(1 + S)
    
    # add additional points to make jumps
    alpha2 = alpha[1:ng] - 0.0000001
    ghat2 = (m - S[0:(ng - 1)])*alpha2/(1 + S[0:(ng - 1)])
    zp = pd.DataFrame({'a': np.concatenate([alpha, alpha2]), 'g': np.concatenate([ghat, ghat2])})
    zp.sort_values(by =['a', 'g'], ascending = [True, False], inplace = True)
    
    # largest gamma hat and index
    gamma_max = np.argmax(zp['g'])
    
    alpha_max = zp['a'][gamma_max]

    # model size with ghat just below gam0
    ind = np.logical_and(ghat <= gam0, alpha <= alpha_max)*1
    Sind = S[np.max(np.where(ind > 0))]
    
    # calculate alpha_F
    alpha_fast = (1 + Sind)*gam0/(m - Sind)
    
    # size of model including intercept
    size1 = sum(pvm <= alpha_fast)
    
    # generate plot
    if plot==True:
        plt.plot(zp['a'], zp['g'], marker = 'o', markersize = 6)
        plt.ylabel('Estimated Gamma')
        plt.xlabel('Alpha')
        pass

    df1 = pd.DataFrame({'pval': pv_orig, 'pvmax': pvm, 'ghigh': ghat2, 'glow': ghat[1:ng]}, columns = ['pval', 'pvmax', 'ghigh', 'glow'])
    df2 = pd.DataFrame({'m1': m1, 'm': m, 'gam0': gam0, 'size': size1, 'alphamax': alpha_max, 'alpha_fast': alpha_fast}, columns = ['m1', 'm', 'gam0', 'size', 'alphamax', 'alpha_fast'], index=[0])
    if printout == True:
        print(df1,df2)
    return(np.round(df1, digits), np.round(df2, digits))

In [130]:
def fsr_fast(x, y, gam0 = 0.05,digits = 4,print = True, plot = True):
    """
    Estimated alpha for forward selection using Fast FSR(no simulation)
    """
    lm = sl.LinearRegression()
    
    df = pd.concat([x,y], axis = 1)
    x = df.ix[:,:-1]
    y = df.ix[:,-1]
    
    # forward selection
    out = reg_subset(x,y)
    pv_orig = out[3]
    m = len(pv_orig)
    n = x.shape[0]
    
    # model selection using fast fsr
    dfs = fsr_fast_pv(pv_orig, m, printout = False, plot = False)
    
    # get the variables should be included
    size = int(dfs[1].ix[0,3])-1
    var_idx = out[0][:size]
    
    # fit model
    if len(var_idx)==0:
        intercept = pd.DataFrame(np.ones(x.shape[0]))
        lm.fit(intercept, y)
        fitted = lm.predict(intercept)
    else:
        x = x.ix[:,var_idx]
        lm.fit(x,y)
        fitted = lm.predict(x)
    residual = sum((y-fitted)**2)
    df_residual = n - m - 1
    return {'fitted':fitted, 'residual':residual, 'df_residual':df_residual, 'size':size, 'index': var_idx}   

In [131]:
fsr_fast(x,y)

{'df_residual': 74,
 'fitted': array([ 38.28433574,  53.16234413,  54.90303156,  54.78940245,
         50.71233174,  52.16487894,  59.09532777,  83.40124575,
         58.02978565,  66.67902977,  53.73664467,  65.18459902,
         55.56477452,  55.95174345,  98.43449241,  45.49718583,
         47.64092308,  60.48879903,  56.8947468 ,  29.83432347,
         67.83320417,  72.37000147,  46.10995947,  40.30341783,
         80.10742078,  64.87192093,  58.47478959,  61.58315636,
         58.02102938,  51.52750406,  52.82426887,  59.43066344,
         39.31562946,  52.78168064,  34.54185136,  65.67886732,
         37.66711133,  62.13517343,  71.17833922,  76.85256553,
         66.13936642,  58.30786786,  52.83021287,  51.13233349,
         71.57216116,  54.50655276,  58.72654417,  56.14774405,
         43.62610782,  71.14995209,  92.2853809 ,  61.64398835,
         59.96780985,  52.71269279,  56.51609532,  45.91619586,
         51.29040743,  69.79272321,  63.69267964,  68.95349892,
         8